<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment2/Assignment2_E1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Experiment 1**

In [1]:
from __future__ import print_function

import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import pandas as pd
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import metrics
import lasagne

Using TensorFlow backend.


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive

In [4]:
import os
os.chdir(os.path.join(os.getcwd(),'drive'))
print(os.getcwd())
print(os.listdir(os.getcwd()))

/content/drive
['.Trash', 'Video_Games_Sales.csv', 'val_data', 'Getting started', 'Track-guidelines.pdf', 'AED_Lab1_Part1.mp4', 'AED_Lab1_Part2Final.mp4', 'games.zip', 'VideoGames-Version2 2.zip', 'VideoGames-Version4.zip', 'VideoGames-Version4-2.zip', 'NBA_stats_f (Recovered).ods', 'RL-NikBrown.odt', 'Resume0102.odt', '6210 Database Design.odt', 'Colab Notebooks', 'out_data_train', 'Imagenet32_train', 'out_data_val', 'x_train.npy', 'y_train.npy', 'x_train_1.npy', 'y_train_1.npy', 'x_val_1.npy', 'y_val_1.npy', 'out_data_train_1', 'out_data_val_1', 'Assignment2 - E 1.ipynb']


In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [0]:
def load_databatch(data_folder, idx, img_size=32):
    data_file = os.path.join(data_folder, 'train_data_batch_')

    d = unpickle(data_file + str(idx))
    x = d['data']
    y = d['labels']
    mean_image = d['mean']

    x = x/np.float32(255)
    mean_image = mean_image/np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i-1 for i in y]
    data_size = x.shape[0]

    x -= mean_image

    img_size2 = img_size * img_size

    x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3))
#     .transpose(0, 3, 1, 2)

    # create mirrored images
    X_train = x[0:data_size, :, :, :]
    Y_train = y[0:data_size]
    X_train_flip = X_train[:, :, :, ::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train, X_train_flip), axis=0)
    Y_train = np.concatenate((Y_train, Y_train_flip), axis=0)

    return dict(
        X_train=lasagne.utils.floatX(X_train),
        Y_train=Y_train.astype('int32'),
        mean=mean_image)

In [0]:
train_data_batch_1 = load_databatch('out_data_train_1', 1, img_size=32)
train_data_batch_2 = load_databatch('out_data_train_1', 2, img_size=32)
train_data_batch_3 = load_databatch('out_data_train_1', 3, img_size=32)
train_data_batch_4 = load_databatch('out_data_train_1', 4, img_size=32)
train_data_batch_5 = load_databatch('out_data_train_1', 5, img_size=32)
train_data_batch_6 = load_databatch('out_data_train_1', 6, img_size=32)
train_data_batch_7 = load_databatch('out_data_train_1', 7, img_size=32)
train_data_batch_8 = load_databatch('out_data_train_1', 8, img_size=32)
train_data_batch_9 = load_databatch('out_data_train_1', 9, img_size=32)
train_data_batch_10 = load_databatch('out_data_train_1', 10, img_size=32)

In [0]:
data1 = train_data_batch_1['X_train']
data2 = train_data_batch_2['X_train']
data3 = train_data_batch_3['X_train']
data4 = train_data_batch_4['X_train']
data5 = train_data_batch_5['X_train']
data6 = train_data_batch_6['X_train']
data7 = train_data_batch_7['X_train']
data8 = train_data_batch_8['X_train']
data9 = train_data_batch_9['X_train']
data10 = train_data_batch_10['X_train']

label1 = train_data_batch_1['Y_train']
label2 = train_data_batch_2['Y_train']
label3 = train_data_batch_3['Y_train']
label4 = train_data_batch_4['Y_train']
label5 = train_data_batch_5['Y_train']
label6 = train_data_batch_6['Y_train']
label7 = train_data_batch_7['Y_train']
label8 = train_data_batch_8['Y_train']
label9 = train_data_batch_9['Y_train']
label10 = train_data_batch_10['Y_train']

In [0]:
x_train = np.concatenate((data1,data2,data3,data4,data5,data6,data7,data8,data9,data10),axis=0)
y_train = np.concatenate((label1,label2,label3,label4,label5,label6,label7,label8,label9,label10),axis=0)

In [0]:
np.save("x_train_1.npy",x_train)

In [0]:
np.save("y_train_1.npy",y_train)

In [0]:
def load_val_data(data, img_size=32):

    d = unpickle(data)
    x = d['data']
    y = d['labels']

    x = x/np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i-1 for i in y]
    data_size = x.shape[0]

    img_size2 = img_size * img_size

    x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3))
#     .transpose(0, 3, 1, 2)

    # create mirrored images
    X_train = x[0:data_size, :, :, :]
    Y_train = y[0:data_size]
    X_train_flip = X_train[:, :, :, ::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train, X_train_flip), axis=0)
    Y_train = np.concatenate((Y_train, Y_train_flip), axis=0)

    return dict(
        X_val=lasagne.utils.floatX(X_train),
        Y_val=Y_train.astype('int32'))

In [0]:
val_data = load_val_data('out_data_val_1/val_data', img_size=32)
x_val = val_data['X_val']
y_val = val_data['Y_val']

In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_Imagenet32_trained_model.h5'

In [0]:
num_classes = 196

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)
    
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [24]:
model_best = Sequential()
model_best.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_best.add(Activation('relu'))
model_best.add(Conv2D(32, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Conv2D(64, (3, 3), padding='same'))
model_best.add(Activation('relu'))
model_best.add(Conv2D(64, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Flatten())
model_best.add(Dense(512))
model_best.add(Activation('relu'))
model_best.add(Dropout(0.1))
model_best.add(Dense(num_classes))
model_best.add(Activation('softmax'))

model_best.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',metrics.categorical_accuracy])

model_best.fit_generator(datagen.flow(x_train, y_train,
    batch_size=32),
    epochs=10,
    steps_per_epoch=1563,
    validation_data=(x_val, y_val),
    workers=4)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1563/1563 [==============================] - 56s 36ms/step - loss: 5.0327 - acc: 0.0275 - categorical_accuracy: 0.0275 - val_loss: 6.6940 - val_acc: 0.0043 - val_categorical_accuracy: 0.0043
Epoch 2/10
1563/1563 [==============================] - 53s 34ms/step - loss: 4.6464 - acc: 0.0653 - categorical_accuracy: 0.0653 - val_loss: 5.9914 - val_acc: 0.0069 - val_categorical_accuracy: 0.0069
Epoch 3/10
1563/1563 [==============================] - 54s 34ms/step - loss: 4.4378 - acc: 0.0905 - categorical_accuracy: 0.0905 - val_loss: 6.6915 - val_acc: 0.0046 - val_categorical_accuracy: 0.0046
Epoch 4/10
1563/1563 [==============================] - 53s 34ms/step - loss: 4.2831 - acc: 0.1112 - categorical_accuracy: 0.1112 - val_loss: 6.5297 - val_acc:

In [25]:
score = model_best.evaluate(x_val, y_val, verbose=1)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

3918/3918 [==============================] - 1s 167us/step
Val loss: 7.158221487246831
Val accuracy: 0.004849412965798877


# **Experiments**

In [0]:
# x_train.shape
# np.save("x_train.npy",x_train)
x_train = np.load("x_train.npy")
x_train = x_train.transpose(0,2,3,1)

In [0]:
y_train = np.load("y_train.npy")

In [0]:
val_data = load_val_data('out_data_val/val_data', img_size=32)

In [0]:
x_val = val_data['X_val']
y_val = val_data['Y_val']

In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_Imagenet32_trained_model.h5'

In [0]:
num_classes = 1000

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)
    
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [15]:
model_best = Sequential()
model_best.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_best.add(Activation('relu'))
model_best.add(Conv2D(32, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Conv2D(64, (3, 3), padding='same'))
model_best.add(Activation('relu'))
model_best.add(Conv2D(64, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Flatten())
model_best.add(Dense(512))
model_best.add(Activation('relu'))
model_best.add(Dropout(0.1))
model_best.add(Dense(num_classes))
model_best.add(Activation('softmax'))

model_best.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',metrics.categorical_accuracy])

model_best.fit_generator(datagen.flow(x_train, y_train,
    batch_size=32),
    epochs=10,
    steps_per_epoch=1563,
    validation_data=(x_val, y_val),
    workers=4)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1563/1563 [==============================] - 61s 39ms/step - loss: 5.0350 - acc: 0.0288 - categorical_accuracy: 0.0288 - val_loss: 6.4525 - val_acc: 0.0056 - val_categorical_accuracy: 0.0056
Epoch 2/10
1563/1563 [==============================] - 55s 35ms/step - loss: 4.5877 - acc: 0.0711 - categorical_accuracy: 0.0711 - val_loss: 6.7511 - val_acc: 0.0063 - val_categorical_accuracy: 0.0063
Epoch 3/10
1563/1563 [==============================] - 53s 34ms/step - loss: 4.3488 - acc: 0.1008 - categorical_accuracy: 0.1008 - val_loss: 6.4371 - val_acc: 0.0035 - val_categorical_accuracy: 0.0035
Epoch 4/10
1563/1563 [==============================] - 51s 33ms/step - loss: 4.1712 - acc: 0.1239 - categorical_accuracy: 0.1239 - val_loss: 6.7610 - val_acc:

In [16]:
score = model_best.evaluate(x_val, y_val, verbose=1)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

3952/3952 [==============================] - 1s 181us/step
Val loss: 7.440697245269652
Val accuracy: 0.005566801619433198
